<a href="https://colab.research.google.com/github/myszojelen23/Emotive-word-matcher-and-calculator/blob/main/Calculating_mean_emotion_values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
from google.colab import files

Importing the Excel sheet with arguments

In [16]:
files.upload()

Saving Argumentacje.xlsx to Argumentacje (1).xlsx


{'Argumentacje.xlsx': b'PK\x03\x04\x14\x00\x06\x00\x08\x00\x00\x00!\x00c\xeb\x9f\x95\xdc\x01\x00\x00\xc8\x08\x00\x00\x13\x00\x08\x02[Content_Types].xml \xa2\x04\x02(\xa0\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [17]:
df = pd.read_excel('Argumentacje.xlsx')

Cleaning

In [18]:
df.rename(columns = {'Imię i nazwisko':'Author', 'Źródło':'Source', 'Typ argumentu (szeregowy lub równoległy)': 'Arg_type', 'Konkluzja':'Conclusion', 'Przesłanka 1':'Premise1', 'Przesłanka 2': 'Premise2', 'Przesłanka 3': 'Premise3', 'Przesłanka 4':'Premise4', 'Przesłanka 5':'Premise5', 'Przesłanka 6':'Premise6', 'Przesłanka 7':'Premise7'}, inplace = True) #renaming columns

df.fillna('', inplace=True)  # Replacing NaN values with string

Joining the text of premises and conclusions in one string



In [19]:
df['Premises'] = df[['Premise1', 'Premise2', 'Premise3', 'Premise4', 'Premise5', 'Premise6', 'Premise7']].agg(' '.join, axis=1)
df['Premises']

df['Arguments'] = df[['Premises','Conclusion']].agg(' '.join, axis=1)

Uploading NAWL database

In [20]:
files.upload()

Saving pone.0132305.s004.xlsx to pone.0132305.s004.xlsx


{'pone.0132305.s004.xlsx': b'PK\x03\x04\x14\x00\x06\x00\x08\x00\x00\x00!\x00b\xee\x9dh^\x01\x00\x00\x90\x04\x00\x00\x13\x00\x08\x02[Content_Types].xml \xa2\x04\x02(\xa0\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

Cleaning NAWL database

In [21]:
nawl = pd.read_excel('pone.0132305.s004.xlsx')

nawl = nawl.drop(columns=['BAWL_word', 'BE_N_all', 'hap_SD_all', 'ang_SD_all', 'sad_SD_all', 'fea_SD_all', 'dis_SD_all', 'BE_N_women', 'hap_M_women',
       'hap_SD_women', 'ang_M_women', 'ang_SD_women', 'sad_M_women',
       'sad_SD_women', 'fea_M_women', 'fea_SD_women', 'dis_M_women',
       'dis_SD_women', 'BE_N_men', 'hap_M_men', 'hap_SD_men', 'ang_M_men',
       'ang_SD_men', 'sad_M_men', 'sad_SD_men', 'fea_M_men', 'fea_SD_men',
       'dis_M_men', 'dis_SD_men', 'DENN_BAWL', 'Briesemeister_liberal',
       'Briesemeister_conservative', 'Mikels', 'Mikels_blended', 'ED_class'])

nawl = nawl.rename(columns = {'No.':'nr', 'NAWL_word':'word', 'hap_M_all':'happiness', 'ang_M_all':'anger', 'sad_M_all':'sadness', 'fea_M_all':'fear', 'dis_M_all':'disgust'})

Lemmatization

In [22]:
!pip install --upgrade spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
!python -m spacy download pl_core_news_sm

2022-08-30 10:34:21.905123: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/explosion/spacy-models/releases/download/pl_core_news_sm-3.4.0/pl_core_news_sm-3.4.0-py3-none-any.whl (20.1 MB)
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_sm')


In [24]:
import spacy

In [25]:
nlp = spacy.load("pl_core_news_sm")

In [26]:
text = df.loc[0,'Arguments'] #this line allows to choose one argument (understood as a string created from the premises and the conclusion) from the database
text_all = df['Arguments'].values.tolist()
text_all = ' '.join(text_all)
print(type(text_all))
#the following line allows to just paste a choosen string
# text = 'Światowa Organizacja Zdrowia (WHO) alarmuje, że osoby palące są bardziej narażone na komplikacje związane zarażeniem koronawirusem SARS-CoV-2. Koronawirus atakuje drogi oddechowe i powoduje ich uszkodzenie. Dym tytoniowy ma negatywny wpływ na stan układu oddechowego. U palaczy mogą wystąpić cięższe dolegliwości związane z zarażeniem koronawirusem. Zrezygnuj z używek.'
doc = nlp(text) #text preprocessing

print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

lemma_list = [] #a list of lemmas contained in each argument - empty so far
for token in doc: #each token contained in the argument...
  lemma_list.append(token.lemma_) #is assigned its dictionary form, lemma

lemma_list

<class 'str'>
Verbs: ['mieć', 'znać być', 'narażać', 'mieć']


['mieć',
 'teraz',
 'do',
 'czynić',
 'z',
 'koronawirus',
 'typ',
 'delt',
 ',',
 'który',
 'nie',
 'znać być',
 'na',
 'początek',
 'pandemia',
 '[',
 'koronawirus',
 'typ',
 'delta',
 ']',
 'to',
 'wyjątkowo',
 'zakaźny',
 'odmiać',
 'i',
 'osoba',
 ',',
 'który',
 'nie',
 'być',
 'zaszczepiyć',
 'narażać',
 'się',
 'na',
 'wyjątkowo',
 'ciężki',
 'przebieg',
 'choroba',
 'teraz',
 'mieć',
 'w',
 'Polska',
 'już',
 'sześć',
 'tysiąc',
 'zakażenie',
 'dziennie',
 'w',
 'województwo',
 'lubelski',
 'szpital',
 'być',
 'już',
 'na',
 'granica',
 'wydolność',
 '   a',
 'szczepma',
 'się']

In [27]:
emo_word = nawl['word'] #A segment of the nawl dataset containing a column with words
emo_word_list = emo_word.tolist() #converted into a list

found_words = [] #empty list for words found

def matcher(argument_list, emo_word_list): #function that matches the words found in the arguments to words in the list of emotive words from the NAWL database; if the word is in the database, it goes into the list
  for word in argument_list:
    if word in emo_word_list:
      found_words.append(word)

matcher(lemma_list, emo_word_list)

found_words


['początek', 'osoba', 'narażać', 'granica']

In [28]:
nawl_fragment = nawl[nawl['word'].isin(found_words)] #A part of the NAWL database that contains only the words included in the given argument
mean_values = nawl_fragment[["happiness", "anger", "sadness", "fear", "disgust"]].mean()
mean_values

happiness    2.7750
anger        2.4850
sadness      2.4825
fear         3.2050
disgust      1.8750
dtype: float64